In [17]:
# install necessary libraries
%pip --quiet install bs4 langchainhub langchain_chroma
%pip --quiet install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# Connect to openAI and get llm model
import os

os.environ["OPENAI_API_KEY"] = ''

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [19]:
# import lib
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader

In [20]:
# Load the PDF file of "The Jaunt" by Stephen King
loader = PyPDFLoader("The_Jaunt-Stephen_King.pdf")
docs = loader.load()

# Split the loaded document into chunks of 1000 characters each, with an overlap of 200 characters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create a vector store from the split documents using OpenAI embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Set up a retriever to find and use the most relevant snippets from the document
retriever = vectorstore.as_retriever()


In [21]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# ------------------custom Chain -----------------------
# First, I create a process that sends the prompt to a large language model (LLM) so it can generate responses.
#question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Next, I set up a process that retrieves tokenized documents.
# This process works together with the question and answer chain I created earlier.
#rag_chain = create_retrieval_chain(retriever, question_answer_chain)
# --------------------------------------------------------------------


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Basic LCEL Chain
rag_chain = (
    {
        "context": vectorstore.as_retriever() | format_docs, #retrieval funtion joined to the promnt statement and the llm question_answer_chain, 
        #wich is custom joined, it is not using the built in funtion, but it is not take the type the document, it only pdf at this point
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("who is patty?")

"Patty is a character in the text provided. She is Mark's daughter, as shown when Mark takes her hand during a conversation about the Jaunt, a teleportation experience. Patty is curious about the Jaunt and asks questions about it, showing interest in the topic."

In [22]:
rag_chain.invoke("What a jaunt?")

'A Jaunt is a form of teleportation in the book "The Jaunt" by Stephen King, where people can travel instantly to different locations. However, Jaunting without being properly sedated results in death or insanity. It has also been used as a murder weapon in some cases, as seen in the story.'